In [1]:
from flask import Blueprint
main = Blueprint('main', __name__)
 
import json
import engine
 
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
 
from flask import Flask, request

#Create User
@main.route("/models/<int:model_id>/users", methods=["POST"])
def create_user(model_id):
    """ body : user_id,products
    """
    user_id = request.form["user_id"]
    products = request.form["products"].split(",")
    user = None
    if model_id == 1:
        user = recommendation_engine_1.add_user(user_id,products)
    elif model_id == 2 :
        user = recommendation_engine_2.add_user(user_id,products)
    elif model_id == 3 :
        user = recommendation_engine_3.add_user(user_id,products)
    outUser = {}
    outUser["user_id"] = user["user_id"]
    outUser["products"] = user["ListItem"]
    return json.dumps(outUser)
 
#Get User
@main.route("/models/<int:model_id>/users/<int:user_id>", methods=["GET"])
def get_user(model_id, user_id):
    logger.debug("User %s product requested", user_id)
    user = None
    if model_id == 1:
        user = recommendation_engine_1.get_user(user_id)
    elif model_id == 2 :
        user = recommendation_engine_2.get_user(user_id)
    elif model_id == 3 :
        user = recommendation_engine_3.get_user(user_id)
    outUser = {}
    outUser["user_id"] = user["user_id"]
    outUser["products"] = user["ListItem"]
    return json.dumps(outUser)

    
#Reccomend a product    
@main.route("/models/<int:model_id>/reccomend/users/<int:user_id>", methods=["GET"])
def reccomend_product(model_id, user_id):
    logger.debug("Reccomending Product for user %s in Model %s", user_id, model_id)
    product = None
    if model_id == 1:
        product = recommendation_engine_1.get_user_reccomendation(user_id)
    elif model_id == 2 :
        product = recommendation_engine_2.get_user_reccomendation(user_id)
    elif model_id == 3 :
        product = recommendation_engine_3.get_user_reccomendation(user_id)
    outItem = {}
    outItem["products"] = product["ListItem"]
    outItem["reccomendation"] = product["prediction"]
    return json.dumps(outItem)

def create_app(spark_context):
    global recommendation_engine_1
    global recommendation_engine_2
    global recommendation_engine_3
 
    recommendation_engine_1 = engine.RecommendationEngine(spark_context, "batch1.csv")
    recommendation_engine_2 = engine.RecommendationEngine(spark_context, "batch2.csv")
    recommendation_engine_3 = engine.RecommendationEngine(spark_context, "batch3.csv")
    
    
    app = Flask(__name__)
    app.register_blueprint(main)
    return app

In [2]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("Product Reccomendation Engine").getOrCreate()
app = create_app(spark)

INFO:engine:Starting Up Reccomendation Engine: 
INFO:engine:Loading Dataset
INFO:engine:Training Dataset
INFO:engine:Starting Up Reccomendation Engine: 
INFO:engine:Loading Dataset
INFO:engine:Training Dataset
INFO:engine:Starting Up Reccomendation Engine: 
INFO:engine:Loading Dataset
INFO:engine:Training Dataset


In [3]:
from paste.translogger import TransLogger
import cherrypy
def run_server(app):
 
    # Enable WSGI access logging via Paste
    app_logged = TransLogger(app)
 
    # Mount the WSGI callable object (app) on the root directory
    cherrypy.tree.graft(app_logged, '/')
 
    # Set the configuration of the web server
    cherrypy.config.update({
        'engine.autoreload.on': True,
        'log.screen': True,
        'server.socket_port': 8091,
        'server.socket_host': '127.0.0.1'
    })
 
    # Start the CherryPy WSGI web server
    cherrypy.engine.start()
    cherrypy.engine.block()

In [ ]:
run_server(app)

[24/May/2019:02:54:53] ENGINE Bus STARTING
INFO:cherrypy.error:[24/May/2019:02:54:53] ENGINE Bus STARTING
[24/May/2019:02:54:53] ENGINE Started monitor thread 'Autoreloader'.
INFO:cherrypy.error:[24/May/2019:02:54:53] ENGINE Started monitor thread 'Autoreloader'.
[24/May/2019:02:54:53] ENGINE Serving on http://127.0.0.1:8091
INFO:cherrypy.error:[24/May/2019:02:54:53] ENGINE Serving on http://127.0.0.1:8091
[24/May/2019:02:54:53] ENGINE Bus STARTED
INFO:cherrypy.error:[24/May/2019:02:54:53] ENGINE Bus STARTED
INFO:engine:User Added
127.0.0.1 - - [24/May/2019:02:55:54 +0700] "POST /models/3/users HTTP/1.1" 200 60 "-" "PostmanRuntime/7.13.0"
INFO:engine:User Added
127.0.0.1 - - [24/May/2019:02:56:12 +0700] "POST /models/2/users HTTP/1.1" 200 60 "-" "PostmanRuntime/7.13.0"
127.0.0.1 - - [24/May/2019:02:56:51 +0700] "GET /models/2/reccomend/users/20222 HTTP/1.1" 200 112 "-" "PostmanRuntime/7.13.0"
127.0.0.1 - - [24/May/2019:03:03:44 +0700] "GET /models/3/reccomend/users/20222 HTTP/1.1" 200 